# Aerial Cactus Identification

This is my attempt at the Kaggle competition entitled Aerial Cactus Identification. The description, taken from [Kaggle](https://www.kaggle.com/), reads as follows: 

 "To assess the impact of climate change on Earth's flora and fauna, it is vital to quantify how human activities such as logging, mining, and agriculture are impacting our protected natural areas. Researchers in Mexico have created the VIGIA project, which aims to build a system for autonomous surveillance of protected areas. A first step in such an effort is the ability to recognize the vegetation inside the protected areas. In this competition, you are tasked with creation of an algorithm that can identify a specific type of cactus in aerial imagery."
 
This project features a bunch of 32x32 images, some containing a columnar cactus. The goal to be predict which do or do not have these cacti. I will attempt this by using a convolutional neural network.

We will begin by loading the packages we will end up using.

In [1]:
import numpy as np
import os
import matplotlib.pyplot as plt
from PIL import Image
import tensorflow as tf
from tensorflow.python.framework import ops
from sklearn.metrics import roc_auc_score
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential

%matplotlib inline

## Loading data

The images must first be loaded into a numpy array so that it can be used in training and evaluating our neural network. 

In [2]:
# Define a function to load the images from a folder into a dictionary
# Keys will be the file names, the images will be an np.array
def load_images(folderpath):
    images = {}
    # Go through file of images, noting their name and loading the images
    for filename in os.listdir(folderpath):
        image = Image.open(os.path.join(folderpath,filename))
        images[filename] = [np.array(image)]
    # Convert images dictionary into a DataFrame
    images_arrays = pd.DataFrame.from_dict(images,orient='index').reset_index().rename({'index':'id'},axis=1)
    return images_arrays
# For the training data, we will also want to include the labels
def load_training_images(labelfile, folderpath):
    images_arrays = load_images(folderpath)
    # Read labels of images into a DataFrame
    images_labels = pd.read_csv(labelfile)
    # Combine labels with the images into a DataFrame
    images_df = images_arrays.merge(images_labels, left_on='id', right_on='id').drop(['id'],axis=1)
    # Return DataFrame with labels and image arrays
    return images_df

In [3]:
# Use the define above function to load our images
training_df = load_training_images('./train.csv', './train')
pred_df = load_images('./test')

# We can split the training dataframe into the image arrays and the labels
X_full = np.stack(training_df.iloc[:,0].values.tolist())
y_full = training_df.iloc[:,1].values.reshape((len(training_df.iloc[:,1]),1))

# We will also want to split up the testing data into the just the image arrays
X_pred_raw = np.stack(pred_df[0])

# Decide on how much to split the training and test sets by
test_size = 0.10
# Split these into training and test sets
X_train_raw, X_test_raw, y_train, y_test = train_test_split(X_full, y_full, test_size=test_size, random_state=42)

In [4]:
# We normalize the data and cast as correct type
X_train = (X_train_raw/255.).astype(np.float32)
X_test = (X_test_raw/255.).astype(np.float32)
y_train = y_train.astype(np.float32)
y_test = y_test.astype(np.float32)
X_pred = (X_pred_raw/255.).astype(np.float32)

# Print out some details of the training/test sets
print("number of training examples = " + str(X_train.shape[0]))
print("number of +ve training examples = " + str(sum(y_train == 1)[0]))
print("number of test examples = " + str(X_test.shape[0]))
print("number of +ve test examples = " + str(sum(y_test == 1)[0]))
print("X_train shape: " + str(X_train.shape))
print("Y_train shape: " + str(y_train.shape))
print("X_test shape: " + str(X_test.shape))
print("Y_test shape: " + str(y_test.shape))
print("X_pred shape: " + str(X_pred.shape))

number of training examples = 15750
number of +ve training examples = 11821
number of test examples = 1750
number of +ve test examples = 1315
X_train shape: (15750, 32, 32, 3)
Y_train shape: (15750, 1)
X_test shape: (1750, 32, 32, 3)
Y_test shape: (1750, 1)
X_pred shape: (4000, 32, 32, 3)


## The Model

Now we will go into defining the model. We will look at a convoluational neural network with two convoluational layers, with activations and max pooling layers in between. Then there is 2 fully connected layers before the final predictive layer. We begin by defining the hyperparameters. 

In [5]:
# CNN Layer filer sizes
cl1fs = 5
cl2fs = 3
# CNN Layer stride
cl1s = 1
cl2s = 1
# CNN Layer output sizes
cl1nc = 8
cl2nc = 16
# Max pooling layer window sizes
ml1ws = 2
ml2ws = 2
# Max pooling layer strides
ml1s = 2
ml2s = 2
# Fully connected layer sizes
f1s = 512
f2s = 256
# Learning rate
learning_rate = 0.001

# Extracted dimensions from above hyperparameters
# Output dim after first max pool
ml1dim = int((32 - ml1ws)/ml1s)+1
# Output dim after second max pool
ml2dim = int((ml1dim-ml2ws)/ml2s)+1
# Output dim after flatten
fladim = ml2dim*ml2dim*cl2nc

In [6]:
# ROC-AUC is the metric of the problem, so we want to use in during training
def auc(y_true, y_pred):
    auc = tf.metrics.auc(y_true, y_pred)[1]
    K.get_session().run(tf.local_variables_initializer())
    return auc

In [7]:
# Build the model using Keras layers
def model_construction():
    model = Sequential()
    # Convolutional layer: channels cl1nc, stride cl1s, padding 'SAME'
    model.add(Conv2D(filters = cl1nc, kernel_size = cl1fs, strides=cl1s, padding='same'))
    # Max pooling: window ml1ws x ml1ws, stride ml1s, padding 'valid'
    model.add(MaxPool2D(pool_size = ml1ws, strides = ml1s, padding = 'valid'))
    # Convolutional layer: channels cl2nc, stride cl2s, padding 'SAME'
    model.add(Conv2D(filters = cl2nc, kernel_size = cl2fs, strides=cl2s, padding='same'))
    # Max pooling: window ml2ws x ml2ws, stride ml2s, padding 'valid'
    model.add(MaxPool2D(pool_size = ml2ws, strides = ml2s, padding = 'valid'))
    # Flatten tensor
    model.add(Flatten())
    # Fully connected layer: size f1s, activation RELU
    model.add(Dense(units = f1s, activation = 'relu'))
    # Fully connected layer size f2s, activation RELU
    model.add(Dense(units = f2s, activation = 'relu'))
    # Fully connected layer size 1 (predictive), activation sigmoid
    model.add(Dense(units = 1, activation = 'sigmoid'))
    # Compile the model with Adams optimizer, crossentropy loss and metric AUC
    model.compile(loss="binary_crossentropy", optimizer='adam', metrics=[auc])
    return model

In [8]:
# Now we train the constructed network to with our training data
model = model_construction()
model.fit(X_train,y_train, validation_data=(X_test,y_test), epochs=200, batch_size=512)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Train on 15750 samples, validate on 1750 samples
Instructions for updating:
Use tf.cast instead.
Epoch 1/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 0.4220 - auc: 0.6959 - val_loss: 0.2810 - val_auc: 0.8464
Epoch 2/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 0.2062 - auc: 0.8888 - val_loss: 0.2103 - val_auc: 0.9158
Epoch 3/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 0.1608 - auc: 0.9297 - val_loss: 0.1466 - val_auc: 0.9411
Epoch 4/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 0.1353 - auc: 0.9486 - val_loss: 0.1276 - val_auc: 0.9547
Epoch 5/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 0.1264 - auc: 0.9591 - val_loss: 0.1108 - val_auc: 0.9628
Epoch

15750/15750 [==============================] - 17s 1ms/sample - loss: 0.0012 - auc: 0.9955 - val_loss: 0.0703 - val_auc: 0.9955
Epoch 52/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 8.3330e-04 - auc: 0.9956 - val_loss: 0.0746 - val_auc: 0.9957
Epoch 53/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 6.6889e-04 - auc: 0.9957 - val_loss: 0.0724 - val_auc: 0.9958
Epoch 54/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 5.3666e-04 - auc: 0.9959 - val_loss: 0.0751 - val_auc: 0.9959
Epoch 55/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 4.6344e-04 - auc: 0.9960 - val_loss: 0.0765 - val_auc: 0.9960
Epoch 56/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 4.2259e-04 - auc: 0.9961 - val_loss: 0.0773 - val_auc: 0.9961
Epoch 57/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 4.2602e-04 - auc: 0.9962 - val_loss: 0.0819 - val_auc: 0.9962
Ep

15750/15750 [==============================] - 17s 1ms/sample - loss: 1.5453e-05 - auc: 0.9982 - val_loss: 0.0940 - val_auc: 0.9982
Epoch 108/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 1.4798e-05 - auc: 0.9983 - val_loss: 0.0945 - val_auc: 0.9983
Epoch 109/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 1.4161e-05 - auc: 0.9983 - val_loss: 0.0942 - val_auc: 0.9983
Epoch 110/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 1.3634e-05 - auc: 0.9983 - val_loss: 0.0940 - val_auc: 0.9983
Epoch 111/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 1.3100e-05 - auc: 0.9983 - val_loss: 0.0948 - val_auc: 0.9983
Epoch 112/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 1.2650e-05 - auc: 0.9983 - val_loss: 0.0944 - val_auc: 0.9983
Epoch 113/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 1.2347e-05 - auc: 0.9983 - val_loss: 0.0948 - val_auc:

15750/15750 [==============================] - 17s 1ms/sample - loss: 2.9321e-06 - auc: 0.9987 - val_loss: 0.1027 - val_auc: 0.9987
Epoch 164/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 2.8891e-06 - auc: 0.9987 - val_loss: 0.1029 - val_auc: 0.9987
Epoch 165/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 2.8194e-06 - auc: 0.9987 - val_loss: 0.1033 - val_auc: 0.9987
Epoch 166/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 2.7545e-06 - auc: 0.9987 - val_loss: 0.1040 - val_auc: 0.9987
Epoch 167/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 2.6589e-06 - auc: 0.9987 - val_loss: 0.1035 - val_auc: 0.9987
Epoch 168/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 2.6250e-06 - auc: 0.9987 - val_loss: 0.1053 - val_auc: 0.9987
Epoch 169/200
15750/15750 [==============================] - 17s 1ms/sample - loss: 2.5580e-06 - auc: 0.9987 - val_loss: 0.1040 - val_auc:

In [9]:
final_predictions = model.predict(X_pred)

In [10]:
# We can then prepare the predictions for submission
submission = (pd.concat([pred_df['id'],pd.DataFrame(final_predictions)],axis=1)
              .rename({0:'has_cactus'},axis=1)
             )
submission

,id,has_cactus
0,000940378805c44108d287872b2f04ce.jpg,1.000000e+00
1,0017242f54ececa4512b4d7937d1e21e.jpg,1.000000e+00
2,001ee6d8564003107853118ab87df407.jpg,0.000000e+00
3,002e175c3c1e060769475f52182583d0.jpg,0.000000e+00
4,0036e44a7e8f7218e9bc7bf8137e4943.jpg,6.207111e-02
5,0038ed6f7417b8275b393307f66cb1e3.jpg,1.000000e+00
6,0059dfa41de4441fc23b9a4b98ba99cb.jpg,1.000000e+00
7,0061bd302d735db829418cb90b4e6040.jpg,1.000000e+00
8,00754573d8c7223e73da553dd43780bd.jpg,1.000000e+00
9,008d9fce71c8e03a4c9cdd15ea908573.jpg,0.000000e+00


In [11]:
submission.to_csv('submission.csv',index=False)